In [22]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

In [23]:
df = pd.read_csv(r'C:\Users\ran14\Workspace\NEWS_TEAM_3 Workspace\models\DATA\Naver_cleasing_total.csv')
pd.set_option('display.max_colwidth', None)
df.head(1)

,platform_id,category,publisher,publication_date,title,content,author,article_url,update_date
0,네이버,금융,부산일보,2024.08.01,"[속보] 몬테네그로 항소법원 “권도형, 한국으로 송환해야”",2022년 국내 투자자 20만 명이 3000억 원에 달하는 피해을 입은 테라루나 사태의 핵심 인물인 테라폼랩스 권도형 대가 한국으로 송환될 가능성이 커졌다 은 지난해 5월 11일 권 씨가 몬테네그로 수도 포드고리차 지방법원으로 향하는 모습 연합뉴스테라루나 폭락 사태의 핵심 인물인 테라폼랩스 권도형 대가 한국 재판대에 오를 가능성이 커졌다1일 몬테네그로 항소법원은 권 씨의 한국 송환을 결정한 포드고리차 고등법원의 판결을 확정했다고 홈페이지를 통해 밝혔다항소법원은 판결문에서 포드고리차 고등법원은 권도형에 대해 한국으로의 약식 인도를 허용한 반면 미국의 범죄인 인도 요청은 기각했다며 이러한 결정에 대해 항소하지 않았으므로 포드고리차 고등법원의 결정은 법적 구속력이 있다고 결정했다,이정훈,https://n.news.naver.com/mnews/article/082/0001282255,수정 날짜 없음


In [24]:
df.head(2).T

,0,1
platform_id,네이버,네이버
category,금융,금융
publisher,부산일보,뉴스1
publication_date,2024.08.01,2024.08.01
title,"[속보] 몬테네그로 항소법원 “권도형, 한국으로 송환해야”","WB '중진국 함정' 극복 대표 사례로 韓 꼽아…""개도국의 필독서"""
content,2022년 국내 투자자 20만 명이 3000억 원에 달하는 피해을 입은 테라루나 사태의 핵심 인물인 테라폼랩스 권도형 대가 한국으로 송환될 가능성이 커졌다 은 지난해 5월 11일 권 씨가 몬테네그로 수도 포드고리차 지방법원으로 향하는 모습 연합뉴스테라루나 폭락 사태의 핵심 인물인 테라폼랩스 권도형 대가 한국 재판대에 오를 가능성이 커졌다1일 몬테네그로 항소법원은 권 씨의 한국 송환을 결정한 포드고리차 고등법원의 판결을 확정했다고 홈페이지를 통해 밝혔다항소법원은 판결문에서 포드고리차 고등법원은 권도형에 대해 한국으로의 약식 인도를 허용한 반면 미국의 범죄인 인도 요청은 기각했다며 이러한 결정에 대해 항소하지 않았으므로 포드고리차 고등법원의 결정은 법적 구속력이 있다고 결정했다,ⓒ News1 정윤미 기자 전민 기자 세계은행은 1일 발한 세계개발보고서에서 중진국 함정을 극복한 대 사례로 한국을 소개했다기획재정부에 따르면 WB는 이날 발한 2024년 세계개발보고서중진국 함정 보고서에서 이처럼 밝혔다WB는 1978년부터 매년 개발 협력 관련 특정 주제를 선정하고 정책적 함의 등을 분석해 세계개발보고서를 발간하고 있다 지난해 주제는 이주난민사회였다WB는 많은 개발도상국이 중진국에 진입한 후 고소득국으로 도약하지 못하고 성장이 정체되는 중진국 함정middleincome trap에 빠지고 있다고 분석하며 한국을 중진국 함정 극복의 대 사례로 제시했다WB는 2022년 1인당 국민총소득 기준 하위 중소득국1136달러4465달러과 상위 중소득국4466달러1만 3845달러을 중진국middle income country으로 그 이상을 고소득국으로 정의했다구체적으로 세계은행은 중진국 함정 극복을 위해 투자Investment 기술 도입 혁신Innovation 3가지가 모두 필요하다는 3i 전략을 제시했다처음 저소득국 단계에서는 투자 촉진으로 성장을 시작하지만 중진국 단계 이후에는 투자 확대만을 통한 성장은 한계에 부딪히고 해외 기술 도입 등을 통한 생산성 제고가 필요하다고 언급했다또한 경제성장을 이끄는 것은 결국 기업가의 열망과 리더십이 만들어 내는 변화라는 조지프 슘페터의 창조적 파괴 이론을 언급했다 지속적인 생산성 제고와 경제성장을 위해서는 낡은 제도와 관습의 창조적 파괴를 통한 혁신이 필요하다고 분석했다WB는 한국이 3i 전략을 효과적으로 활용해 1인당 GNI가 1960년 1200달러 이하에서 2023년 약 3만3000달러 수준으로 급격하게 성장한 성장 슈퍼스타Super Star라고 언급했다 또 한국 경제 발전사는 개도국 정책 입안자의 필독서required reading라고 소개했다한국의 성장 배경으로는 금융시장 개방과 외국 자본 유치 등을 통한 인프라 투자 확대 해외 기술의 도입과 연구개발 교육 등에 대한 적극적인 투자를 통한 생산성 제고 등을 꼽았다WB는 보고서에서 최근 지정학적 긴장으로 인한 무역과 투자의 위축 포퓰리즘과 공공부채의 증가 기후변화 등이 중진국 성장 전망을 더욱 어둡게 하고 있다고 경고했다 그러면서 오늘날 중진국이 한국이 25년 만에 이뤄낸 성과를 50년 만에 달성하는 것도 기적이라고 해도 지나치지 않은 상황이라고 언급했다 WB는 한국 이외에 고성장을 이룬 국가로 폴란드 칠레 등을 언급했다기획재정부 관계자는 세계은행이라는 전 세계적으로 권위 있는 국제기구가 한국의 성장 역사를 극찬하고 이를 바탕으로 개도국의 성장 전략을 제시하는 것은 매우 고무적이라고 평가했다이어 보고서에서 제시한 3i 전략의 혁신 생태계 조성 사회 이동성 공정경쟁 등 주요한 내용들이 모두 우리 정부가 발한 역동경제 로드맵에 포함된 내용으로 일맥상통한다고 덧붙였다
author,이정훈,전민 기자
article_url,https://n.news.naver.com/mnews/article/082/0001282255,https://n.news.naver.com/mnews/article/421/0007704171
update_date,수정 날짜 없음,2024.08.01


# Feature Selection

In [71]:
news_df = df[['title', 'content', 'article_url']]
news_df.head(2)

,title,content,article_url
0,"[속보] 몬테네그로 항소법원 “권도형, 한국으로 송환해야”",2022년 국내 투자자 20만 명이 3000억 원에 달하는 피해을 입은 테라루나 사태의 핵심 인물인 테라폼랩스 권도형 대가 한국으로 송환될 가능성이 커졌다 은 지난해 5월 11일 권 씨가 몬테네그로 수도 포드고리차 지방법원으로 향하는 모습 연합뉴스테라루나 폭락 사태의 핵심 인물인 테라폼랩스 권도형 대가 한국 재판대에 오를 가능성이 커졌다1일 몬테네그로 항소법원은 권 씨의 한국 송환을 결정한 포드고리차 고등법원의 판결을 확정했다고 홈페이지를 통해 밝혔다항소법원은 판결문에서 포드고리차 고등법원은 권도형에 대해 한국으로의 약식 인도를 허용한 반면 미국의 범죄인 인도 요청은 기각했다며 이러한 결정에 대해 항소하지 않았으므로 포드고리차 고등법원의 결정은 법적 구속력이 있다고 결정했다,https://n.news.naver.com/mnews/article/082/0001282255
1,"WB '중진국 함정' 극복 대표 사례로 韓 꼽아…""개도국의 필독서""",ⓒ News1 정윤미 기자 전민 기자 세계은행은 1일 발한 세계개발보고서에서 중진국 함정을 극복한 대 사례로 한국을 소개했다기획재정부에 따르면 WB는 이날 발한 2024년 세계개발보고서중진국 함정 보고서에서 이처럼 밝혔다WB는 1978년부터 매년 개발 협력 관련 특정 주제를 선정하고 정책적 함의 등을 분석해 세계개발보고서를 발간하고 있다 지난해 주제는 이주난민사회였다WB는 많은 개발도상국이 중진국에 진입한 후 고소득국으로 도약하지 못하고 성장이 정체되는 중진국 함정middleincome trap에 빠지고 있다고 분석하며 한국을 중진국 함정 극복의 대 사례로 제시했다WB는 2022년 1인당 국민총소득 기준 하위 중소득국1136달러4465달러과 상위 중소득국4466달러1만 3845달러을 중진국middle income country으로 그 이상을 고소득국으로 정의했다구체적으로 세계은행은 중진국 함정 극복을 위해 투자Investment 기술 도입 혁신Innovation 3가지가 모두 필요하다는 3i 전략을 제시했다처음 저소득국 단계에서는 투자 촉진으로 성장을 시작하지만 중진국 단계 이후에는 투자 확대만을 통한 성장은 한계에 부딪히고 해외 기술 도입 등을 통한 생산성 제고가 필요하다고 언급했다또한 경제성장을 이끄는 것은 결국 기업가의 열망과 리더십이 만들어 내는 변화라는 조지프 슘페터의 창조적 파괴 이론을 언급했다 지속적인 생산성 제고와 경제성장을 위해서는 낡은 제도와 관습의 창조적 파괴를 통한 혁신이 필요하다고 분석했다WB는 한국이 3i 전략을 효과적으로 활용해 1인당 GNI가 1960년 1200달러 이하에서 2023년 약 3만3000달러 수준으로 급격하게 성장한 성장 슈퍼스타Super Star라고 언급했다 또 한국 경제 발전사는 개도국 정책 입안자의 필독서required reading라고 소개했다한국의 성장 배경으로는 금융시장 개방과 외국 자본 유치 등을 통한 인프라 투자 확대 해외 기술의 도입과 연구개발 교육 등에 대한 적극적인 투자를 통한 생산성 제고 등을 꼽았다WB는 보고서에서 최근 지정학적 긴장으로 인한 무역과 투자의 위축 포퓰리즘과 공공부채의 증가 기후변화 등이 중진국 성장 전망을 더욱 어둡게 하고 있다고 경고했다 그러면서 오늘날 중진국이 한국이 25년 만에 이뤄낸 성과를 50년 만에 달성하는 것도 기적이라고 해도 지나치지 않은 상황이라고 언급했다 WB는 한국 이외에 고성장을 이룬 국가로 폴란드 칠레 등을 언급했다기획재정부 관계자는 세계은행이라는 전 세계적으로 권위 있는 국제기구가 한국의 성장 역사를 극찬하고 이를 바탕으로 개도국의 성장 전략을 제시하는 것은 매우 고무적이라고 평가했다이어 보고서에서 제시한 3i 전략의 혁신 생태계 조성 사회 이동성 공정경쟁 등 주요한 내용들이 모두 우리 정부가 발한 역동경제 로드맵에 포함된 내용으로 일맥상통한다고 덧붙였다,https://n.news.naver.com/mnews/article/421/0007704171


In [72]:
news_df['content'][1]

'ⓒ News1 정윤미 기자 전민 기자 세계은행은 1일 발한 세계개발보고서에서 중진국 함정을 극복한 대 사례로 한국을 소개했다기획재정부에 따르면 WB는 이날 발한 2024년 세계개발보고서중진국 함정 보고서에서 이처럼 밝혔다WB는 1978년부터 매년 개발 협력 관련 특정 주제를 선정하고 정책적 함의 등을 분석해 세계개발보고서를 발간하고 있다 지난해 주제는 이주난민사회였다WB는 많은 개발도상국이 중진국에 진입한 후 고소득국으로 도약하지 못하고 성장이 정체되는 중진국 함정middleincome trap에 빠지고 있다고 분석하며 한국을 중진국 함정 극복의 대 사례로 제시했다WB는 2022년 1인당 국민총소득 기준 하위 중소득국1136달러4465달러과 상위 중소득국4466달러1만 3845달러을 중진국middle income country으로 그 이상을 고소득국으로 정의했다구체적으로 세계은행은 중진국 함정 극복을 위해 투자Investment 기술 도입 혁신Innovation 3가지가 모두 필요하다는 3i 전략을 제시했다처음 저소득국 단계에서는 투자 촉진으로 성장을 시작하지만 중진국 단계 이후에는 투자 확대만을 통한 성장은 한계에 부딪히고 해외 기술 도입 등을 통한 생산성 제고가 필요하다고 언급했다또한 경제성장을 이끄는 것은 결국 기업가의 열망과 리더십이 만들어 내는 변화라는 조지프 슘페터의 창조적 파괴 이론을 언급했다 지속적인 생산성 제고와 경제성장을 위해서는 낡은 제도와 관습의 창조적 파괴를 통한 혁신이 필요하다고 분석했다WB는 한국이 3i 전략을 효과적으로 활용해 1인당 GNI가 1960년 1200달러 이하에서 2023년 약 3만3000달러 수준으로 급격하게 성장한 성장 슈퍼스타Super Star라고 언급했다 또 한국 경제 발전사는 개도국 정책 입안자의 필독서required reading라고 소개했다한국의 성장 배경으로는 금융시장 개방과 외국 자본 유치 등을 통한 인프라 투자 확대 해외 기술의 도입과 연구개발 교육 등에 대한 적극적인 투자를 통한 생산성 제고 등을 꼽

# Tokenization

In [73]:
from ekonlpy.tag import Mecab
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [74]:
# mecab으로 content 컬럼 토큰화

mecab = Mecab()

documents = [TaggedDocument(words=mecab.morphs(row['content']), tags=[str(i)]) for i, row in news_df.iterrows()]

In [79]:
documents

[TaggedDocument(words=['2022', '년', '국내', '투자자', '20', '만', '명', '이', '3000', '억', '원', '에', '달하', '는', '피해', '을', '입', '은', '테라', '루나', '사태', '의', '핵심', '인물', '인', '테라', '폼', '랩스', '권도형', '대', '가', '한국', '으로', '송환', '될', '가능성', '이', '커졌', '다', '은', '지난해', '5', '월', '11', '일', '권', '씨', '가', '몬테네그로', '수도', '포드고리차', '지방', '법원', '으로', '향하', '는', '모습', '연합뉴스', '테라', '루나', '폭락', '사태', '의', '핵심', '인물', '인', '테라', '폼', '랩스', '권도형', '대', '가', '한국', '재', '판대', '에', '오를', '가능성', '이', '커졌', '다', '1', '일', '몬테네그로', '항소', '법원', '은', '권', '씨', '의', '한국', '송환', '을', '결정', '한', '포드고리차', '고등', '법원', '의', '판결', '을', '확정', '했', '다고', '홈페이지', '를', '통해', '밝혔', '다', '항소', '법원', '은', '판결문', '에서', '포드고리차', '고등', '법원', '은', '권도', '형', '에', '대해', '한국', '으로', '의', '약식', '인도', '를', '허용', '한', '반면', '미국', '의', '범죄', '인', '인도', '요청', '은', '기각', '했', '다며', '이러', '한', '결정', '에', '대해', '항소', '하', '지', '않', '았', '으므로', '포드고리차', '고등', '법원', '의', '결정', '은', '법', '적', '구속력', '이', '있', '다고', '결정', '했', '다'], tags=['0']),


In [76]:
model = Doc2Vec(documents, vector_size=100, window=5, min_count=1, workers=4, epochs=10)

In [77]:
news_df.loc[:, 'content_vector'] = news_df['content'].apply(lambda x: model.infer_vector(x.split()))

C:\Users\ran14\AppData\Local\Temp\ipykernel_23112\2875187988.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df.loc[:, 'content_vector'] = news_df['content'].apply(lambda x: model.infer_vector(x.split()))


In [80]:
news_df

,title,content,article_url,content_vector
0,"[속보] 몬테네그로 항소법원 “권도형, 한국으로 송환해야”",2022년 국내 투자자 20만 명이 3000억 원에 달하는 피해을 입은 테라루나 사태의 핵심 인물인 테라폼랩스 권도형 대가 한국으로 송환될 가능성이 커졌다 은 지난해 5월 11일 권 씨가 몬테네그로 수도 포드고리차 지방법원으로 향하는 모습 연합뉴스테라루나 폭락 사태의 핵심 인물인 테라폼랩스 권도형 대가 한국 재판대에 오를 가능성이 커졌다1일 몬테네그로 항소법원은 권 씨의 한국 송환을 결정한 포드고리차 고등법원의 판결을 확정했다고 홈페이지를 통해 밝혔다항소법원은 판결문에서 포드고리차 고등법원은 권도형에 대해 한국으로의 약식 인도를 허용한 반면 미국의 범죄인 인도 요청은 기각했다며 이러한 결정에 대해 항소하지 않았으므로 포드고리차 고등법원의 결정은 법적 구속력이 있다고 결정했다,https://n.news.naver.com/mnews/article/082/0001282255,"[-0.49807897, -0.03634523, -0.059111442, 0.009672909, -0.077875905, 0.0028434969, 0.13649361, 0.316316, -0.29106718, -0.23045397, 0.3068635, 0.13921463, 0.4311536, 0.096804336, -0.35558918, -0.5714489, -0.10290798, 0.012767648, -0.19252515, 0.014558635, -0.15098782, -0.17606434, -0.06614737, -0.015283728, 0.46414402, 0.32296744, -0.5977414, 0.19477247, -0.16032878, 0.24784392, 0.14148304, 0.041748513, 0.107633516, 0.22412781, 0.12700787, 0.31288442, -0.52285975, -0.007845731, -0.19577967, 0.1670541, -0.23550992, 0.58092123, -0.74736965, 0.08283817, 0.047602657, -0.016364263, -0.30719015, 0.23228616, 0.25329936, 0.2670862, 0.34563163, -0.10657288, -0.1607222, -0.43084183, 0.122249454, 0.28737423, 0.097598836, -0.09283615, 0.07833759, 0.05641381, -0.17634259, -0.19267292, -0.048682947, -0.054314725, -0.025647907, 0.44536963, 0.36206743, -0.008141037, 0.0072243977, 0.07573001, -0.128014, -0.24322993, 0.4818695, -0.21869707, 0.64395666, 0.1109883, -0.3224694, -0.12962033, -0.10934183, 0.12707786, -0.25783285, -0.035716232, 0.31027135, 0.10520157, -0.030695044, -0.06602297, 0.24245669, 0.06796516, -0.36627808, 0.42770892, 0.19448598, -0.1967016, 0.449953, 0.025372906, -0.034823515, 0.4691767, -0.38092837, 0.11346105, -0.1929085, 0.13935055]"
1,"WB '중진국 함정' 극복 대표 사례로 韓 꼽아…""개도국의 필독서""",ⓒ News1 정윤미 기자 전민 기자 세계은행은 1일 발한 세계개발보고서에서 중진국 함정을 극복한 대 사례로 한국을 소개했다기획재정부에 따르면 WB는 이날 발한 2024년 세계개발보고서중진국 함정 보고서에서 이처럼 밝혔다WB는 1978년부터 매년 개발 협력 관련 특정 주제를 선정하고 정책적 함의 등을 분석해 세계개발보고서를 발간하고 있다 지난해 주제는 이주난민사회였다WB는 많은 개발도상국이 중진국에 진입한 후 고소득국으로 도약하지 못하고 성장이 정체되는 중진국 함정middleincome trap에 빠지고 있다고 분석하며 한국을 중진국 함정 극복의 대 사례로 제시했다WB는 2022년 1인당 국민총소득 기준 하위 중소득국1136달러4465달러과 상위 중소득국4466달러1만 3845달러을 중진국middle income country으로 그 이상을 고소득국으로 정의했다구체적으로 세계은행은 중진국 함정 극복을 위해 투자Investment 기술 도입 혁신Innovation 3가지가 모두 필요하다는 3i 전략을 제시했다처음 저소득국 단계에서는 투자 촉진으로 성장을 시작하지만 중진국 단계 이후에는 투자 확대만을 통한 성장은 한계에 부딪히고 해외 기술 도입 등을 통한 생산성 제고가 필요하다고 언급했다또한 경제성장을 이끄는 것은 결국 기업가의 열망과 리더십이 만들어 내는 변화라는 조지프 슘페터의 창조적 파괴 이론을 언급했다 지속적인 생산성 제고와 경제성장을 위해서는 낡은 제도와 관습의 창조적 파괴를 통한 혁신이 필요하다고 분석했다WB는 한국이 3i 전략을 효과적으로 활용해 1인당 GNI가 1960년 1200달러 이하에서 2023년 약 3만3000달러 수준으로 급격하게 성장한 성장 슈퍼스타Super Star라고 언급했다 또 한국 경제 발전사는 개도국 정책 입안자의 필독서required reading라고 소개했다한국의 성장 배경으로는 금융시장 개방과 외국 자본 유치 등을 통한 인프라 투자 확대 해외 기술의 도입과 연구개발 교육 등에 대한 적극적인 투자를 통한 생산성 제고 등을 꼽았다WB는 보고서에서 최근 지정학적 긴장으로 인한 무역과 투자의 위축 포퓰리즘과 공공부채의 증가 기후변화 등이 중진국 성장 전망을 더욱 어둡게 하고 있다고 경고했다 그러면서 오늘날 중진국이 한국이 25년 만에 이뤄낸 성과를 50년 만에 달성하는 것도 기적이라고 해도 지나치지 않은 상황이라고 언급했다 WB는 한국 이외에 고성장을 이룬 국가로 폴란드 칠레 등을 언급했다기획재정부 관계자는 세계은행이라는 전 세계적으로 권위 있는 국제기구가 한국의 성장 역사를 극찬하고 이를 바탕으로 개도국의 성장 전략을 제시하는 것은 매우 고무적이라고 평가했다이어 보고서에서 제시한 3i 전략의 혁신 생태계 조성 사회 이동성 공정경쟁 등 주요한 내용들이 모두 우리 정부가 발한 역동경제 로드맵에 포함된 내용으로 일맥상통한다고 덧붙였다,https://n.news.naver.com/mnews/article/421/0007704171,"[-0.71786314, 0.25898778, -1.0686338, 0.6334822, -0.87593657, -0.20833544, -0.17456028, 0.80250305, 0.095159024, -0.7372045, 0.7637072, -0.16271637, 0.11860177, 0.15901342, -0.1902951, -0.038819257, 0.7298642, 0.43907925, -0.50913906, -0.1548683, 0.3062474, 0.18818055, -0.33188835, -0.175082, 0.57740146, 0.64208144, -0.7436215, 0.044421114, 0.08376475, -0.20886712, -0.29483533, 0.12797458, -0.13399865, 0.2184109, -0.016817667, 0.32681158, -0.15758173, -0.8640607, -0.25042543, 0.41652888, 0.4144281, -0.14544675, -0.36767152, -0.11255404, 0.56660384, 0.036110673, -0.02377347, -0.20945883, 0.3224467, -0.09550743, -0.26998442, -0.1243718, -1.1607981, 0.17498077, 0.2721655, 1.0379997, -0.47615227, 0.28317556, 0.24812807, 0.2

1. title, url, content feature selection
2. 일단 코사인 유사도 계산까지
3. 유저가 댓글을 본 기사의 수는 어떤것이 가장 효과적인가

# sample data 작업

In [81]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# 1. 데이터 샘플링
sampled_news_df = news_df.sample(n=1000, random_state=42).reset_index(drop=True) 


In [82]:
sampled_news_df

,title,content,article_url,content_vector
0,[그래픽] SK하이닉스 상반기 순차입금 15.5조…부채 규모↓,16일 SK하이닉스 반기보고서에 따르면 올해 2분기 말 기준 SK하이닉스의 순차입금은 15조5300억원이다 순차입금은 차입금외부에서 빌린 자금에서 현금 및 현금성자산을 뺀 금액으로 수치가 클수록 재무 부담이 큰 것으로 볼 수 있다,https://n.news.naver.com/mnews/article/003/0012731913,"[-0.41044155, -0.077646196, 0.028723016, -0.018352417, -0.09769567, -0.34809062, 0.06655142, -0.14763243, 0.05872169, -0.1667216, -0.15368187, -0.02874189, 0.25462562, 0.21332142, -0.019045517, -0.15073328, 0.1252762, 0.25748613, 0.15201376, -0.055687204, -0.061778225, -0.10135981, -0.034531698, -0.009650901, 0.11390254, -0.028228614, -0.10002121, 0.16941707, 0.082658574, -0.18760633, 0.0667685, -0.0645561, -0.13666075, 0.060073894, -0.03383282, -0.031352565, -0.08725515, -0.034003112, -0.1734119, -0.026653532, 0.045416698, -0.23662136, 0.031102095, -0.20285448, 0.33583143, -0.035009746, 0.023637539, -0.029382695, 0.028114604, -0.19576362, -0.0037906782, -0.004498466, -0.31854907, 0.22020914, -0.027147297, 0.0059489147, -0.039169148, -0.3264852, 0.001150062, -0.03398538, -0.13200669, -0.0818209, -0.005776339, 0.07930922, -0.057082966, 0.056425627, 0.048758157, -0.1361031, -0.129752, -0.07181936, -0.030212972, 0.13951148, 0.017095033, 0.080652654, 0.109670974, -0.007968142, 0.028668752, -0.15222599, -0.027546905, 0.07219755, 0.13044363, -0.15596727, -0.0444418, 0.03653022, -0.14836103, 0.044735413, 0.14413297, 0.28513023, -0.16149624, 0.30469364, -0.07887109, -0.08323958, 0.08653931, 0.16144532, -0.014755946, -0.062184066, 0.09848484, -0.008353673, 0.26101965, 0.014535902]"
1,갈 길은 먼데...美 지표 기다리는 시장 [마켓인사이트],여러 경제 지 발를 앞두고 지난밤 뉴욕증시에는 관망 분위기가 짙게 깔렸습니다뉴욕 연은의 소비자 기대 인플레이션이 다소 큰폭으로 하락하면서 9월 금리 인하 가능성을 뒷받침했습니다美 1년 기대인플레 2972020년 11월 이후 최저치美 3년 기대인플레 23311년래 최저이란의 이스라엘 타격 가능성에 국제유가는 4대 급등했습니다*WTI 7684달러 8006달러 419↑13일 마켓인사이트에서는 김홍석 KCGI자산운용 주식운용본부장과 시장 분석해봤습니다<한국경제TV 마켓인사이트>는 증시가 열리는 매일 오전 11시 생방송으로 진행됩니다 투자에 대한 조언일 뿐 판단은 여러분들의 몫입니다● 코스피도 숨고르기코스피 2600선 회복했지만 반등이 지속될지 확신은 부족합니다이날 코스피 역시 장중 오르락 내리락하더니 소폭 상승으로 마감했습니다 3거래일 연속 오름세입니다*한국증시 마감 코스피 320p 상승 262150코스닥 786p 하락 76486 김홍석 본부장은 미국의 전반적인 경기 침체에 대한 불안감이 여전히 남아있다면서 주식시장이 급반등은 했지만 반등세를 계속 진행이 계속되기는 쉽지 않아 보인다고 내다봤습니다 이번 증시 폭락 전후 미국 경제의 펀더멘털이 달라진 점이 없다는 점을 강조했습니다우리 시간으로 내일 밤 발될 미국의 7월 소비자물가지수가 급격하게 둔화될 경우 경기 침체 우려도 다시 불거질 수 있습니다김홍석 본부장은 CPI가 시장 예상과 비슷하거나 다소 조금 높게 나오는 것이 주식시장에는 오히려 더 긍정적인 요인이라면서도 최근 소비 둔화 추세로 볼때 시장 예상보다 낮게 나올 것으로 예상했습니다*주요 이벤트 美 7월 생산자물가지수 美 7월 소비자물가지수 日 2분기 GDP 中 7월 실업률 美 7월 소매판매산업생산 美 주간 신규실업수당 청구건수 美 필라델피아연준 제조업지수 美 8월 미시건대 소비심리평가지수 ● 빅컷 증시에는 악재 연준의 빅컷 필요성을 주장하는 목소리도 여전합니다김홍석 본부장은 빅컷은 시장에 아직 알려지지 않은 뭔가 두려움을 느낄 만한 이벤트들이 있다는 것을 시장에 얘기해주는 강력한 시그널이라며 주식시장에는 굉장히 큰 악재로 큰 낙폭을 불러 일으킬 수 있다고 봤습니다 그러면서 연준이 9월과 12월 025p씩 2차례 인하하는게 주식시장에는 가장 좋은 그림이라고 전망했습니다엔 캐리 트레이드 청산 등 여러가지 요인들이 시장을 짓누르고 있지만 투자 전략은 결국 미국의 경기 흐름에 집중해서 봐야한다고 조언했습니다 미국 경제가 고성장의 끝자락 경기 침체 초입에 있다고 봤을 때 보수적 투자 스탠스가 현명하다고 했습니다국내 유망섹터로는 수출경쟁력이 높은 기업과 산업을 꼽았고 반도체에 대해서는 34년 정도를 놓고 보면 여전히 회복기라고 분석했습니다,https://n.news.naver.com/mnews/article/215/0001175166,"[-1.6457508, 0.13390534, -0.17986357, -0.052404243, -0.13958785, -0.781229, -0.40082076, 0.7383188, -0.64328647, 0.4589075, 0.68556505, 0.339315, 0.6059241, -0.36255893, 0.49970913, 0.024705594, 0.30510423, 0.11531627, -0.3943918, 0.46948674, 0.6691007, 0.47754928, 0.34336364, 0.33354294, 0.16645214, 0.17660317, 0.15991913, -0.3153275, -0.028358627, -0.21843514, 0.67472637, -0.45488694, 0.20835607, -0.22298892, -0.13844666, 1.0938952, -0.33007482, 0.16957206, -0.96818674, -0.73000956, 0.47779065, -0.43033835, 0.35839194, 0.008447854, 0.7846912, 0.5824456, 0.48900524, -0.91571385, 0.18791658, -0.9791484, 0.017974403, 0.5769901, -0.3166823, 0.21603686, -0.35444257, 0.65595514, -0.28963462, -0.18257476, -0.056800127, -0.18736368, -0.3752231, -0.10340827, -0.0012474338, 0.069512695, -1.0376426, -0.3602655, -0.08852428, -0.63359535, -0.65156937, 0.38352165, -0.71137124, 0.22917369, 0.089734115, 0.35581002, 0.34840187, 0.5581902, 0.040712275, -0.4219071, 0.36762366, -0.31368452, 0.12454534, 0.011695827, 0.28239444, 0.334

In [63]:
# # 2. Doc2Vec 준비 (이미 content 벡터화가 완료되었다고 가정)
# documents = [TaggedDocument(words=row['content'].split(), tags=[str(i)]) for i, row in sampled_news_df.iterrows()]
# model = Doc2Vec(documents, vector_size=100, window=5, min_count=1, workers=4, epochs=40)

# # 3. 각 기사의 content를 벡터화
# sampled_news_df['content_vector'] = sampled_news_df['content'].apply(lambda x: model.infer_vector(x.split()))

In [64]:
# 4. 코사인 유사도 계산
content_vectors = np.array(sampled_news_df['content_vector'].tolist())
cosine_sim_matrix = cosine_similarity(content_vectors)

In [83]:
# 5. 특정 title과 유사한 기사 상위 10개 추출하는 함수
def get_similar_articles(article_url, top_n=10):
    # 입력받은 title과 일치하는 기사의 인덱스 찾기
    idx = sampled_news_df[sampled_news_df['article_url'] == article_url].index[0]
    
    # 해당 기사의 유사도 벡터 가져오기
    sim_scores = cosine_sim_matrix[idx]
    
    # 유사도 순으로 정렬된 상위 top_n개의 기사 인덱스 가져오기
    similar_idx = np.argsort(-sim_scores)[1:top_n+1]  # 자기 자신을 제외하기 위해 [1:]부터
    
    # 유사한 기사 상위 top_n개 반환
    return sampled_news_df.iloc[similar_idx][['title', 'content']]

# 6. 함수 실행: 예시 title에 유사한 상위 10개의 기사 출력
similar_articles = get_similar_articles('https://n.news.naver.com/mnews/article/469/0000817606')
print(similar_articles)


                                               title  \
321           "1인 가구인데, 오래된 '청약통장' 유지해야 하나요?" [혼자인家]   
67   “100억 넘어도 고급주택 아니라니”...북촌가옥 세금폭탄, 청담아파트는 피해갔다는데   
917                 총회 한번에 1억7천만원 쓰는 재건축조합…85%가 서면투표   
497               "배정 초등학교 바꿔달라" 울산 신축 아파트 학부모 소송 기각   
902                  "똥 배관까지 터졌다"…광양 신축 아파트 '처참한 상황'   
619              ‘월 1200만원’ 벌던 배달기사, 교통사고로 한 달 만에 숨져   
712        "햇살론 대환대출 믿고 맡겼는데..." 불법사금융 피해 5년새 1.5배 ↑   
654                       카드 월세납부 증가 ‘짠테크족’ 수수료도 따진다   
583       하동서 추락 산림청 헬기 'AS350'…2011년에도 사고 2명 사망(종합)   
7            "25분 만에 4인분 먹거리 뚝딱"... 신일전자, 대용량 두유 제조기   

                                                                                                                                                                                                                                                                                                                                                                                               

# 원본 데이터 작업